In [ ]:
!pip install faiss-cpu transformers torch numpy tqdm sentence-transformers faiss-gpu 
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu118

In [5]:
import logging

# 设置日志
logging.basicConfig(filename='chatbot.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s:%(message)s')

def chat(query, tokenizer, model, device, index, cleaned_documents, top_k=5):
    retrieved_docs = retrieve_documents(query, tokenizer, model, device, index, cleaned_documents, top_k)
    answer = generate_answer(query, retrieved_docs, tokenizer, model, device)
    # 记录日志
    logging.info(f"Query: {query}")
    logging.info(f"Retrieved Docs: {retrieved_docs}")
    logging.info(f"Answer: {answer}")
    return answer

In [ ]:
import os
import re
import faiss
import torch
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import StoppingCriteria, StoppingCriteriaList

# 1. 加载和预处理文档
def load_documents(directory_path, chunk_size=500, overlap=50):
    """
    加载文档并将其拆分为多个块。

    Args:
        directory_path (str): 文档目录路径。
        chunk_size (int): 每个块的字符数。
        overlap (int): 相邻块之间的重叠字符数。

    Returns:
        list: 拆分后的文档块列表。
    """
    documents = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                content = content.replace('\n', ' ').strip()
                if content:
                    # 使用滑动窗口拆分
                    for i in range(0, len(content), chunk_size - overlap):
                        chunk = content[i:i + chunk_size]
                        chunk = chunk.strip()
                        if chunk:
                            documents.append(chunk)
    return documents

def clean_text(text):
    """
    清洗文本，去除多余的空格和特殊字符。

    Args:
        text (str): 原始文本。

    Returns:
        str: 清洗后的文本。
    """
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\u4e00-\u9fa5A-Za-z0-9\s\.,，。!?]', '', text)
    return text

# 2. 生成嵌入向量
def get_embedding(text, embedding_model):
    """
    生成文本的嵌入向量。

    Args:
        text (str): 输入文本。
        embedding_model (SentenceTransformer): 句子嵌入模型。

    Returns:
        np.ndarray: 归一化后的嵌入向量。
    """
    embedding = embedding_model.encode(text, normalize_embeddings=True)
    return embedding

# 3. 检索相关文档块
def retrieve_documents(query, embedding_model, index, cleaned_documents, top_k=2, threshold=0.3):
    """
    根据查询检索最相关的文档块，并设置阈值过滤不相关的文档。

    Args:
        query (str): 用户查询。
        embedding_model (SentenceTransformer): 句子嵌入模型。
        index (faiss.Index): FAISS索引。
        cleaned_documents (list): 清洗后的文档块列表。
        top_k (int): 检索的相关文档块数量。
        threshold (float): 文档相关性的最低阈值。

    Returns:
        list: 检索到的相关文档块列表。
    """
    query_embedding = get_embedding(query, embedding_model)
    query_embedding = np.expand_dims(query_embedding, axis=0).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    
    retrieved_docs = []
    for i, dist in zip(indices[0], distances[0]):
        if dist >= threshold:
            retrieved_docs.append(cleaned_documents[i])
    
    # 保证最多返回 3 个文档块
    retrieved_docs = retrieved_docs[:3]
    
    # 打印检索结果
    print("检索到的文档块：")
    for i, doc in enumerate(retrieved_docs):
        print(f"文档块 {i+1}: {doc[:200]}...")  # 仅显示前200字符

    return retrieved_docs

# 4. 生成回答
class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        if input_ids[0][-1] == self.stop_token_id:
            return True
        return False

def generate_answer(query, retrieved_docs, tokenizer, model, device, max_new_tokens=256):
    """
    根据查询和检索到的文档块生成回答。

    Args:
        query (str): 用户查询。
        retrieved_docs (list): 检索到的相关文档块。
        tokenizer (AutoTokenizer): 生成模型的分词器。
        model (AutoModelForCausalLM): 生成模型。
        device (torch.device): 设备（CPU或GPU）。
        max_new_tokens (int): 生成回答的最大新令牌数。

    Returns:
        str: 生成的回答。
    """
    # 限制每个文档块的长度，避免上下文过长
    truncated_docs = []
    for doc in retrieved_docs:
        truncated_doc = doc[:500]  # 取每个文档块的前500个字符
        truncated_docs.append(truncated_doc)

    context = "\n---\n".join(truncated_docs)  # 使用分隔符区分不同文档块
    input_text = f"问题：{query}\n相关文档：\n{context}\n回答："

    # 编码输入
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=1024)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # 设置 pad_token_id，如果模型有 pad_token_id，则使用它；否则使用 eos_token_id
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

    # 定义自定义停止条件
    stopping_criteria = StoppingCriteriaList([StopOnToken(tokenizer.eos_token_id)])

    # 生成回答，仅设置 max_new_tokens，移除 max_length
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_return_sequences=1,
            do_sample=True,  # 如果希望增加多样性，可以设置为 True
            num_beams=3,  # 设置 beam 数量
            pad_token_id=pad_token_id,  # 明确设置 pad_token_id
            eos_token_id=tokenizer.eos_token_id,  # 设置 eos_token_id
            no_repeat_ngram_size=2,  # 防止重复
            early_stopping=True,  # 启用早停
            stopping_criteria=stopping_criteria  # 使用自定义停止条件
        )

    # 解码生成的文本
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 提取回答部分（假设回答以"回答："开头）
    if "回答：" in answer:
        answer = answer.split("回答：")[-1].strip()
    return answer

# 5. 整合聊天流程
def chat(query, embedding_model, tokenizer, model, device, index, cleaned_documents, top_k=2, threshold=0.3):
    """
    处理用户查询并生成回答。

    Args:
        query (str): 用户查询。
        embedding_model (SentenceTransformer): 句子嵌入模型。
        tokenizer (AutoTokenizer): 生成模型的分词器。
        model (AutoModelForCausalLM): 生成模型。
        device (torch.device): 设备（CPU或GPU）。
        index (faiss.Index): FAISS索引。
        cleaned_documents (list): 清洗后的文档块列表。
        top_k (int): 检索的相关文档块数量。

    Returns:
        str: 生成的回答。
    """
    retrieved_docs = retrieve_documents(query, embedding_model, index, cleaned_documents, top_k, threshold)
    answer = generate_answer(query, retrieved_docs, tokenizer, model, device)
    return answer

def main():
    # 设置路径
    knowledge_base_dir = '/kaggle/input/myragtxt'  # 替换为您的知识库目录路径
    faiss_index_path = '/kaggle/working/faiss_index_ivf.index'
    # 替换为实际的本地生成模型路径
    local_model_path = '/kaggle/input/lora/pytorch/default/1' 
    #local_model_path = '/kaggle/input/qwen2.5/transformers/3b-instruct/1' 
    
    
    # 1. 加载和预处理文档
    print("加载文档中...")
    # 设置文档分块参数
    chunk_size = 1000  # 每个块的字符数
    overlap = 200       # 块之间的重叠字符数
    documents = load_documents(knowledge_base_dir, chunk_size=chunk_size, overlap=overlap)
    print(f"已加载并分块 {len(documents)} 个文档块。")
    
    print("清洗文档中...")
    cleaned_documents = [clean_text(doc) for doc in documents]
    
    # 2. 加载嵌入模型和生成模型
    print("加载嵌入模型...")
    # 使用适合中文的 Sentence-Transformer 模型
    embedding_model = SentenceTransformer('shibing624/text2vec-base-chinese')  # 你可以根据需要选择其他模型
    
    print("加载生成模型和分词器...")
    # 加载生成模型和分词器
    tokenizer = AutoTokenizer.from_pretrained(local_model_path, use_fast=False)
    model = AutoModelForCausalLM.from_pretrained(local_model_path)
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # 3. 生成嵌入向量
    print("生成文档块嵌入向量中...")
    # 使用批量编码提高效率
    document_embeddings = embedding_model.encode(cleaned_documents, normalize_embeddings=True, show_progress_bar=True)
    document_embeddings = np.array(document_embeddings).astype('float32')
    print(f"已生成 {len(document_embeddings)} 个文档块的嵌入向量。")
    
    # 4. 构建 FAISS 索引
    print("构建 FAISS 索引中...")
    dimension = document_embeddings.shape[1]
    
    if len(document_embeddings) == 0:
        print("没有文档块可供索引。程序退出。")
        return
    elif len(document_embeddings) < 100:
        # 如果文档块数量少于100，使用 IndexFlatIP
        print("文档块数量少于100。使用 IndexFlatIP 索引。")
        index = faiss.IndexFlatIP(dimension)
        index.add(document_embeddings)
    else:
        # 否则，使用 IndexIVFFlat 并设置合适的 nlist
        nlist = min(100, max(100, len(document_embeddings) // 10))  # 一般 nlist = num_clusters
        quantizer = faiss.IndexFlatIP(dimension)  # 内积量度
        index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)
        # 训练索引
        print("训练 FAISS 索引中...")
        index.train(document_embeddings)
        print("FAISS 索引已训练。")
        # 添加向量到索引
        index.add(document_embeddings)
    
    # 保存索引
    faiss.write_index(index, faiss_index_path)
    print(f"FAISS 索引已保存到 {faiss_index_path}。")
    # 如果需要加载索引，可以使用以下代码
    # index = faiss.read_index(faiss_index_path)
    
    # 5. 测试聊天功能
    print("欢迎使用 Qwen 聊天机器人！输入内容即可开始对话。\n输入 \\quit 结束会话。\n")
    while True:
        # 用户输入
        user_query = input("用户: ").strip()
        
        # 退出会话
        if user_query.lower() == r"\quit":
            print("聊天机器人已退出，会话结束。")
            break
        
        # 输出用户输入
        print(f"用户: {user_query}")
        
        # 获取助手的回应
        response = chat(user_query, embedding_model, tokenizer, model, device, index, cleaned_documents, top_k=1)
        
        # 输出助手回应
        print(f"助手: {response}\n")

if __name__ == "__main__":
    main()
